# Dependencies

In [1]:
import MURA
import cv2
import image_manipulation
from multiprocessing import Pool
from models import *
import numpy as np
import glob
import matplotlib.pyplot as plt
import preprocessing
import json
from os import path, mkdir

import keras
from argparse import ArgumentParser
import tensorflow as tf

c:\Users\SKill\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model to Run
This section lets the user edit on what model parameters to run, the model directory and parameters should exist prior to changing the file path

In [2]:
# All editable variables
model_file_path = "models/model_2"
# If preprocessing needs to run
run_preprocessing = False

In [3]:
# Get model parameters
try:
    params = json.load(open(model_file_path + '/parameters.json'))

    # Model to use
    model_is_VAE = params['is_VAE']
    # Model parameters
    multiplier = params['multiplier']
    latent_size = params['latent_size']
    input_shape = params['input_shape']

    # Training parameters
    epochs = params['num_epochs']
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']

    # Dataset Path
    image_paths = MURA.MURA_DATASET()
    dataset_file_path = params['dataset_path']
    all_image_paths = image_paths.get_combined_image_paths()
    all_image_paths = all_image_paths.to_numpy()[:,0]
except:
    raise Exception("No parameters.json file found in the model's directory.")

In [4]:
# Do preprocessing
if run_preprocessing:
    preprocess = preprocessing.preprocessing(input_path = all_image_paths, output_path = dataset_file_path)
    if __name__ == '__main__':
        preprocess.start()

In [5]:
# each array contains the training, validation, and testing in order
image_datasets = {'train': [],
                'valid': [],
                'test': []}
for dataset_name in image_datasets.keys():
    for image_path in glob.glob(f'{dataset_file_path}/{dataset_name}/*.png'):
        image_datasets[dataset_name].append(cv2.imread(image_path))
    image_datasets[dataset_name] = np.array(image_datasets[dataset_name])

# Model Training
This section creates and trains the model

In [6]:
if __name__ == "__main__":

    if model_is_VAE:
        model = VAE(False, input_shape, multiplier, latent_size)
    else:
        model = UPAE(True, input_shape, multiplier, latent_size)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.build(input_shape=(None, 64, 64, 3))

    model.compile(optimizer= optimizer
                  ,metrics=[tf.keras.metrics.Accuracy()])
    
    # Where images of each epoch will be saved
    save_directory = 'Images/images_epochs'
    save_callback = SaveImageCallback(image_datasets['train'], save_directory)

    # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
    #training on training set.
    history_train = model.fit(image_datasets['train'], 
                epochs=epochs, 
                batch_size=batch_size,
                validation_split=0.15,
                callbacks=[save_callback])
    

Model: "encoder_decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 128)      512       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 16, 16, 128)   

# Model Prediction

In [7]:
history_valid = model.predict(image_datasets['valid'], batch_size=batch_size)

In [11]:
history_valid[0][0]

<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
array([[[0.00000000e+00, 0.00000000e+00, 1.81942768e+01],
        [0.00000000e+00, 0.00000000e+00, 2.48510246e+01],
        [0.00000000e+00, 0.00000000e+00, 2.96470337e+01],
        ...,
        [0.00000000e+00, 0.00000000e+00, 9.54594727e+01],
        [0.00000000e+00, 0.00000000e+00, 9.52634735e+01],
        [0.00000000e+00, 0.00000000e+00, 5.97324219e+01]],

       [[0.00000000e+00, 0.00000000e+00, 2.51535339e+01],
        [1.77127242e-01, 0.00000000e+00, 2.86279335e+01],
        [0.00000000e+00, 2.68575788e-01, 2.53165588e+01],
        ...,
        [1.35485995e+00, 0.00000000e+00, 1.05796829e+02],
        [0.00000000e+00, 0.00000000e+00, 9.98215637e+01],
        [0.00000000e+00, 0.00000000e+00, 9.49383392e+01]],

       [[0.00000000e+00, 0.00000000e+00, 2.66872368e+01],
        [0.00000000e+00, 0.00000000e+00, 2.50604095e+01],
        [0.00000000e+00, 0.00000000e+00, 2.83864632e+01],
        ...,
        [0.00000000e+00, 0.0000000

# Plots Creation

In [ ]:
# train_loss = history_train.history['binary_crossentropy: ']
# # valid_loss = history_valid.history['binary_crossentropy: ']

# plt.plot(train_loss , label='train')
# # plt.plot(valid_loss , label='test')
# plt.title('Train vs Validation Loss')
# plt.ylabel('Reconstruction Loss')
# plt.xlabel('Epochs')
# plt.legend()
# plt.show()

# Testing of the Model with the Test Set
This section tests the model with the current test set
TODO: 
- Get the label of each image in the test set
- Test the images
- Create Linear Regression for the abnormality score to get the threshold for determining abnormal or normal images

# Saving of final reconstructed images 

In [12]:
# Create directory in models folder for reconstructed images
dataset_name = dataset_file_path.split('/')[-1]
reconstructed_images_path = model_file_path + "/" + dataset_name
if not path.exists(reconstructed_images_path):
    mkdir(reconstructed_images_path)

In [14]:
for x in range(len(history_valid)):
    fig, axs = plt.subplots(1,2, figsize=(8,4))
    axs[0].imshow(image_datasets['valid'][x])
    axs[0].set_title('Original Image')
    new_image = np.floor(history_valid[0][x]).astype(np.uint8)
    axs[1].imshow(new_image)
    axs[1].set_title('Reconstructed Image')
    plt.savefig(f'{reconstructed_images_path}/Valid_Image_{x}.png')
    plt.close()
    break

# Saving of Model Weights

In [ ]:
model.save_weights(model_file_path + '/model_weights.h5')

In [ ]:
history_valid[0].shape

In [ ]:
new_image = np.concatenate(history_valid[0], axis=1)
plt.imshow(new_image)

In [ ]:
history_valid[0]